# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from ACD import train_ACD_model
import numpy as np
import constants
import warnings
import torch

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_term" # "aspect_term", "aspect_category"
LLM_SAMPLING = "fixed"

## Settings

In [3]:
# Set seeds
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

device(type='mps')

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING)

### Load Model

In [5]:
if TARGET == "aspect_category":
   results = train_ACD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

Some weights of the model checkpoint at deepset/gbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/32 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5188, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.4075073301792145, 'eval_hamming_loss': 0.1088, 'eval_accuracy': 0.456, 'eval_f1_macro': 0.537800966793195, 'eval_f1_micro': 0.7269076305220884, 'eval_f1_weighted': 0.6741813541045737, 'eval_class_f1_scores': [0.4071856287425149, 1.0, 0.7781155015197568, 0.5037037037037037, 0.0], 'eval_runtime': 6.9316, 'eval_samples_per_second': 72.134, 'eval_steps_per_second': 4.617, 'epoch': 1.0}
{'train_runtime': 39.6608, 'train_samples_per_second': 12.607, 'train_steps_per_second': 0.807, 'train_loss': 0.5188348293304443, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
if TARGET == "aspect_term":